In [1]:
! pip install ruwordnet
! ruwordnet download

  Created wheel for ruwordnet: filename=ruwordnet-0.0.3-py3-none-any.whl size=11621 sha256=3fbfb6e87961efa85d5cb2d72af3eb92810c354be9cf68502763514dffa3aa0e
  Stored in directory: /root/.cache/pip/wheels/44/82/4b/016af6cfab5855b87d3015eee1650f5560ee9fab885c7ef6b3
Successfully built ruwordnet
downloading a ruwordnet model from https://github.com/avidale/python-ruwordnet/releases/download/0.0.2/ruwordnet.db


In [2]:
! pip install wiktionaryparser

In [3]:
import re

In [4]:
from tqdm.auto import tqdm
tqdm.pandas()

WordNet

In [5]:
from ruwordnet import RuWordNet
import pandas as pd

wn = RuWordNet()

Wiktionary

In [6]:
from wiktionaryparser import WiktionaryParser

wp = WiktionaryParser()
wp.set_default_language('Russian')
entries = wp.fetch('лук')
for entry in entries:
    print(entry)

{'etymology': 'Inherited from Proto-Slavic *lukъ, borrowed from Proto-Germanic *laukaz (compare German Lauch, English leek, Old Norse laukr whence Danish løg). Cognates include Polabian lāuk, Bulgarian лук (luk) and Slovene and Serbo-Croatian luk.\n', 'definitions': [{'partOfSpeech': 'noun', 'text': ['лук • (luk)\xa0m\xa0inan (genitive лу́ка, nominative plural лу́ки, genitive plural лу́ков, related adjective лу́ковый)', '(usually uncountable, collectively) onion, onions'], 'relatedWords': [], 'examples': ['зелёный лук ― zeljónyj luk ― scallion', 'лук-поре́й ― luk-poréj ― leek']}], 'pronunciations': {'text': ['IPA: [ɫuk]', 'Homophone: луг (lug)'], 'audio': ['//upload.wikimedia.org/wikipedia/commons/4/49/Ru-%D0%BB%D1%83%D0%BA.ogg']}}
{'etymology': 'Inherited from Proto-Slavic *lǫkъ. Cognate with Lithuanian lankas.\n', 'definitions': [{'partOfSpeech': 'noun', 'text': ['лук • (luk)\xa0m\xa0inan (genitive лу́ка, nominative plural лу́ки, genitive plural лу́ков)', 'bow (weapon used for shooti

Частотный словарь Ляшевской и Шаровой

In [7]:
! unzip Freq2011.zip

Archive:  Freq2011.zip
  inflating: freqrnc2011.csv         
  inflating: freqrnc_readme.txt      


In [8]:
freqrnc = pd.read_csv('freqrnc2011.csv', sep='\t')
freqrnc = freqrnc[freqrnc.PoS == 's']

НКРЯ

In [9]:
from smart_open import open

In [10]:
def get_lemmas_count_rnc(filepath):

    with open(filepath, mode='rt') as f:
        rows = f.readlines()

    table = []
    for row in rows:
        row = row.strip().split(';')[:11]
        table.append(tuple(row))

    df = pd.DataFrame(table[1:], columns=table[0])
    df = df[df.Cat == 'S']

    grouped_df = df.groupby('Lemma').agg("count")
    grouped_df = grouped_df[['Cat']].sort_values(['Cat'], ascending=False).reset_index()
    grouped_df['Lemma'] = grouped_df['Lemma'].astype(str)
    grouped_df['Lemma'] = grouped_df['Lemma'].apply(lambda x: x.lower())
    lemmas_data = grouped_df[~grouped_df.Lemma.str.contains("[a-z0-9]")].rename({'Cat':'rnc_num_senses'}, axis=1)

    return lemmas_data

In [13]:
lemmas = get_lemmas_count_rnc('semantic.csv.gz')

In [14]:
# объединить частотность и количество значений в НКРЯ
freq_and_count = pd.merge(freqrnc, lemmas, on='Lemma', how='inner')

In [15]:
# 200 самых частотных существительных
most_freq_lemmas = freq_and_count.sort_values('Freq(ipm)')[-1000:]

In [16]:
def fetch_with_catch_error(x):
    try:
        return len([1 for i in wp.fetch(x)])
    except AttributeError as E:
        print(x, E)

def ruwordnet_wiki_comparison(data):
    
    data['wiktionary_num_senses'] = data['Lemma'].progress_apply(fetch_with_catch_error)
    data['ruwordnet_num_senses'] = data['Lemma'].progress_apply(lambda x: len(wn.get_senses(x)))
    
    return data

In [17]:
result = ruwordnet_wiki_comparison(most_freq_lemmas)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
result.reset_index(inplace=True, drop=True)

In [19]:
result['Mean'] = result[['rnc_num_senses', 'wiktionary_num_senses', 'ruwordnet_num_senses']].mean(axis=1).round().astype('int')

In [20]:
result = result.drop(['R', 'D', 'Doc'], axis=1).reset_index(drop=True)

In [21]:
result.head()

,Lemma,PoS,Freq(ipm),rnc_num_senses,wiktionary_num_senses,ruwordnet_num_senses,Mean
0,единица,s,52.4,5,1,1,2
1,чемпионат,s,52.4,2,1,1,1
2,строка,s,52.5,2,1,2,2
3,затрата,s,52.5,3,1,2,2
4,покупатель,s,52.6,1,1,1,1


In [22]:
def groups(num):
    c = None
    if num == 1:
        c = 1
    elif num <= 4:
        c = 2
    else:
        c = 3
    return c

In [23]:
result['Group'] = result.Mean.apply(groups)

In [24]:
import scipy.stats  as stats
stats.pearsonr(result[:200]['Freq(ipm)'], result[:200]['Mean'])

(-0.04251681813498022, 0.5499866291259291)

In [25]:
stats.pearsonr(result[:200]['Freq(ipm)'], result[:200]['Group'])

(-0.06424966413781474, 0.3660662238271576)

In [26]:
one = result[result.Group == 1]
few = result[result.Group == 2]
many = result[result.Group == 3]

In [27]:
target_words = pd.concat([one[-8:], few[-8:], many[-8:]], ignore_index=True).sort_values(['Freq(ipm)'], ascending=False, ignore_index=True)

In [28]:
target_words.head()

,Lemma,PoS,Freq(ipm),rnc_num_senses,wiktionary_num_senses,ruwordnet_num_senses,Mean,Group
0,год,s,3727.5,5,1,1,2,2
1,человек,s,2723.0,4,1,2,2,2
2,время,s,2015.7,9,1,5,5,3
3,дело,s,1412.1,11,2,4,6,3
4,жизнь,s,1389.8,6,1,1,3,2


In [29]:
target_words.to_csv('target_words.tsv', sep='\t', index=False)